# Jagged reduction

Given a jagged array, we want to compute a reduction (e.g. `sum` or `max`) as quickly as possible. We'll study the naive sequential algorithm, Jaydeep's modified Hillis-Steele on GPU and AVX-512. As a further modification, we replace the offsets-driven gather with a parents-driven scatter, since the extra pass gather was found to be dominant over the reduction itself.

## First step: prepare data

The jagged datasets are Gaussian random numbers in Poisson-distributed groups, a different dataset for each Poisson-average group size. To make debugging easier, the Gaussian numbers will be 1 +- 0.01 (the sum will always be approximately equal to the count), and there will be 15 samples with logarithmically spaced Poisson-averages:

    0.3 0.5 1.0 2.0 3.0 5.0 10.0 20.0 30.0 50.0 100.0 200.0 300.0 500.0 1000.0

This is about 15 GB, and we'll cache them on disk (in `/tmp/DATA`).

In [1]:
import numpy
import os

In [1]:
def make_dataset(averagesize, tasksize=100000000, location="/tmp/DATA"):
    if not os.path.exists(location):
        os.mkdir(location)
    counts = numpy.random.poisson(averagesize, int(numpy.ceil(tasksize / averagesize)))
    offsets = numpy.empty(len(counts) + 1, dtype=numpy.int32)
    offsets[0] = 0
    numpy.cumsum(counts, out=offsets[1:])
    del counts
    with open(os.path.join(location, "offsets-{}".format(averagesize)), "wb") as f:
        offsets.tofile(f)

    parents = numpy.zeros(len(content), dtype=numpy.int32)
    numpy.add.at(parents, offsets[offsets != offsets[-1]][1:], 1)
    numpy.cumsum(parents, out=parents)
    with open(os.path.join(location, "parents-{}".format(averagesize)), "wb") as f:
        parents.tofile(f)
    del parents
    
    content = numpy.random.normal(1, 0.01, offsets[-1]).astype(numpy.float32)
    with open(os.path.join(location, "content-{}".format(averagesize)), "wb") as f:
        content.tofile(f)
    del content
    del offsets

# for averagesize in (0.3, 0.5, 1.0, 2.0, 3.0, 5.0, 10.0, 20.0, 30.0,
#                     50.0, 100.0, 200.0, 300.0, 500.0, 1000.0):
#     make_dataset(averagesize)

In [7]:
def get_dataset(averagesize, location="/tmp/DATA"):
    offsets = numpy.fromfile(open(os.path.join(location, "offsets-{}".format(averagesize))),
                             dtype=numpy.int32)
    parents = numpy.fromfile(open(os.path.join(location, "parents-{}".format(averagesize))),
                             dtype=numpy.int32)
    content = numpy.fromfile(open(os.path.join(location, "content-{}".format(averagesize))),
                             dtype=numpy.float32)
    return offsets, parents, content

## Sequential algorithm

This is the natural algorithm, stepping through jagged subcollections in a doubly nested for loop. We also use it to define the "correct" output. We'll study two such algorithms, `sum` and `max`.

In [4]:
%load_ext Cython

In [ ]:
%%cython --cplus -c-std